# Model template using multiple polygons
This is an exercise to test the computing time for fitting and predicting spatial models using an assortment of different polygons.

This can work as a general template for other models and data pipelines.

In [ ]:
%matplotlib inline
import sys
sys.path.append('/apps')
import django
django.setup()
from drivers.tree_builder import TreeNeo
from drivers.graph_models import TreeNode, Order, Family, graph,Kingdom,Occurrence
from drivers.graph_models import Cell,Mex4km, countObjectsOf
from drivers.graph_models import pickNode
import traversals.strategies as st
from os import walk
import matplotlib.pyplot as plt
import pandas as pd
import itertools as it
import numpy as np
import pymc3 as pm

## Use the ggplot style
plt.style.use('ggplot')

In [ ]:
def loadDataset(path_to_dataset):
    _files = []
    for (dirpath, dirnames,filenames) in walk(path_to_dataset):
        _files = map(lambda f : dirpath + '/' + f ,filenames)
    #print(_files)
    ## Read all data
    dataset = map(lambda f : pd.read_csv(f,na_values=["N.A.","NaN","N.A"],encoding='utf8'),_files)  
    dataset = map(lambda d : st.toGeoDataFrame(d,xcoord_name='Longitude',ycoord_name='Latitude'),dataset)    
    return dataset

In [ ]:
bursera_path = '/outputs/presence_only_models/data/burseras'
bursera_dataset = loadDataset(bursera_path)

train_path = '/outputs/presence_only_models/data/root'
train_dataset = loadDataset(train_path)
## Predictors
pred_path = '/outputs/presence_only_models/predictors/datasetp1'
pred_dataset = loadDataset(pred_path)
### PATCH, the thing is taking backwards the order of the lists of files, because of the name
pred_dataset.reverse()

In [ ]:
## Recreating the polygons
from django.contrib.gis.geos import Point, Polygon
xcoord = -99.76
ycoord = 17.55
p = Point(xcoord,ycoord,srid=4326)
radii = np.linspace(0.08,1, 10)
polys = map(lambda r : p.buffer(r),radii)


## Obtaining the predictors
In this case we will bring all the variables to start working with everything

In [ ]:
from raster_api.tools import RasterData
from raster_api.models import raster_models_dic as models


In [ ]:
### Preprocessing of CSV files
def preparePredictors(pred_dataset):
    """
    Prepares the predictor datasets.
    Parameters : 
        pred_dataset : A list of dataframes obtained from the predictor function 
        i.e. compileDataCube (for the moment hold in notebook, obtaining_predictors)
    Returns :
        list of geopandas dataframes.
    
    """
    datapred = pred_dataset
    datapred = datapred.replace("N.A",np.nan)
    ## Remove NAs from coordinates (necessary, given the hereogeneous datasource (included in worldpop and elevation)
    datapred.dropna(subset=['Longitude','Latitude'],inplace=True)
    ## Remove problematic string columns, so that all of them can be numeric
    #datapred = datapred.drop(['vegname','inegiv5name'],axis=1)
    ## Convert to numeric
    ## Very nice way to convert to numeric 
    #cols = datapred.columns.drop('Unnamed: 0')
    #datapred[cols] = datapred[cols].apply(pd.to_numeric,errors='coerce')
    ## New data set without nas for any other column
    datacube_clean = datapred.dropna()
    ## Convert to geopandas
    #datacube_clean = st.toGeoDataFrame(datacube_clean,xcoord_name='Longitude',ycoord_name='Latitude')
    return {'clean' : datacube_clean, 'full' :datapred}
    #return datacube_clean

In [ ]:
prediction_dataset_dic= map(lambda p : preparePredictors(p),pred_dataset)


In [ ]:
#map(lambda p : p['full'].plot(column='DistanceToRoadMex'),prediction_dataset_dic)

In [ ]:
## Convert to numeric
## Very nice way to convert to numeric 
#cols = datapred.columns.drop('Unnamed: 0')
#datapred[cols] = datapred[cols].apply(pd.to_numeric,errors='coerce')
## New data set without nas for any other column
#datacube_clean = datapred.dropna()
## Convert to geopandas
#from external_plugins.spystats.spystats import tools as tl
#datacube_clean = tl.toGeoDataFrame(datacube_clean,xcoord_name='Longitude',ycoord_name='Latitude')


### Ok, lets start the ,modelling here

In [ ]:
## just a small subsample
#datafull = datatrain.replace('N.A.',np.nan).astype('float')
#datafull = datasets[9].dropna()
#Y = datafull.Burseraceae
i = 1
datatrain = train_dataset[i]
Y = datatrain.Burseraceae
datapred = prediction_dataset_dic[i]
polygon = polys[i]

In [ ]:
#pm.traceplot(trace)
from raster_api.tools import RasterContainer
from raster_api.models import ETOPO1,MeanTemperature
from raster_api.tools import RasterData
from sketches.models import Country
from mesh.models import MexMesh

In [ ]:
## This is for calculating the signal
from pymc3.variational.callbacks import CheckParametersConvergence
def FitMyModel(Y,train,predictor):
    #
    with pm.Model() as model:

        ## [R | Y]

        tau = pm.HalfNormal('tau',sd=10)
        sigma = pm.HalfNormal('sigma',sd=10)
        phi = pm.Uniform('phi',0,15)

        Tau = pm.gp.cov.Constant(tau)
        cov = (sigma * pm.gp.cov.Matern32(2,phi,active_dims=[0,1])) + Tau

        ## Parameters for linear predictor
        #b0 = pm.Normal('b0',mu=0,sd=10)
        b = pm.Normal('b',mu=0,sd=10,shape=3)
        mf = pm.gp.mean.Linear(coeffs=[b]) 

        ## The latent function
        gp = pm.gp.Latent(cov_func=cov)
        f = gp.prior("latent_field", X=train[['Longitude','Latitude','DistanceToRoadMex_mean','WorldPopLatam2010_mean','vegid']].values,reparameterize=False)


        ## Other model M2

        beta_y = pm.Normal("betay",mu=0, sd=10,shape=2)

        theta = beta_y[0] + beta_y[1] * train.MaxTemperature_mean.values

        yy = pm.Bernoulli("yy",logit_p=theta,observed=Y.values)


        #y_obs = pm.Bernoulli('y_obs',logit_p=(f*yy),observed=Y.values)


        #trace = pm.fit(method='advi', callbacks=[CheckParametersConvergence()],n=15000)    
        trace = pm.sample(300)
        #trace = trace.sample(draws=5000)


        f_star = gp.conditional("f_star", predictor['clean'][['Longitude','Latitude','DistanceToRoadMex','WorldPopLatam2010','vegid']].values)

        pred_samples = pm.sample_ppc(trace, vars=[f_star], samples=100)
        return pred_samples
   

In [ ]:
def plotThings(pred_samples):
    preds = pd.DataFrame(pred_samples['f_star']).transpose()
    import scipy.special as sp
    alpha = sp.logit(0.5)
    mean_sample = preds.mean(axis=1)
    q_025 = preds.quantile(0.025,axis=1)
    q_975 = preds.quantile(0.975,axis=1)
    prob_gt05 = preds.apply(lambda row : float(sum(row > alpha))/100,axis=1)
    surface_data = pd.DataFrame({'mean_sample' : mean_sample, 'q_025':q_025,'q_975':q_975,'prob_gt05':prob_gt05})

    #preds['idx'] = data_star.index.values
    surface_data['idx'] = datapred['clean'].index.values
    predictions = datapred['full'].merge(surface_data,how='left',left_index=True,right_on='idx',suffixes=('_obs','_pred'))
    predicted_data = predictions.mean_sample.values
    # Raster Container
    ql_presences_of_something = RasterContainer(predictions.q_025.values,use_metadata_from=elevation.rasterdata,exponential_fam="Bernoulli")
    ql_presences_of_something.display_field(band=2,origin='Lower',cmap=plt.cm.RdBu,interpolation='None',title="Quantiles 0.025")

    mean_presences_of_something = RasterContainer(predicted_data,use_metadata_from=elevation.rasterdata,exponential_fam="Bernoulli")
    mean_presences_of_something.display_field(band=2,origin='Lower',cmap=plt.cm.RdBu,interpolation='None',title="mean probability")

    qh_presences_of_something = RasterContainer(predictions.q_975.values,use_metadata_from=elevation.rasterdata,exponential_fam="Bernoulli")
    qh_presences_of_something.display_field(band=2,origin='Lower',cmap=plt.cm.RdBu,interpolation='None',title="quantile 0.097")

    ## Probability of the probaaility of presences bigger than 0.5
    prob5 = RasterContainer(predictions.prob_gt05.values,use_metadata_from=elevation.rasterdata,exponential_fam="Bernoulli")
    prob5.display_field(band=2,origin='Lower',cmap=plt.cm.RdBu,interpolation='None',title="probability of exceding more than 0.5")



In [ ]:
for i in range(len(train_dataset)):
    #datatrain = bursera_dataset[i]
    datatrain = train_dataset[i]
    #Y = datatrain.Burseraceae
    Y = datatrain.Burseraceae
    datapred = prediction_dataset_dic[i]
    polygon = polys[i]
    polygon_predict = polygon
    elevation = RasterData(rastermodelinstance=ETOPO1,border=polygon)
    pixel_size = 0.01
    elevation.rescale(pixel_size)
    #elevation.display_field(origin='Lower',interpolation='None')
    %time pred_samples = FitMyModel(Y,datatrain,datapred)
    plotThings(pred_samples)

In [ ]:
from scipy.special import expit
fig, ax = plt.subplots(figsize=(10, 9));

#ncounts_families.display_field(band=2,origin='Lower',title='family richness')
expit(ql_presences_of_something.rasterdata.bands[0].data())
plt.imshow(ql_presences_of_something.rasterdata.bands[1].data(),origin='Lower',cmap=plt.cm.Greens)
plt.title("Presences quantile 0.025")
plt.colorbar(orientation='horizontal')

In [ ]:
from scipy.special import expit
fig, ax = plt.subplots(figsize=(10, 9));

#ncounts_families.display_field(band=2,origin='Lower',title='family richness')
plt.imshow(mean_presences_of_something.rasterdata.bands[1].data(),origin='Lower',cmap=plt.cm.Greens,clim=(0,1))

plt.colorbar(orientation='horizontal')
plt.title('Probability of presences (mean surface)' )

In [ ]:
from scipy.special import expit
fig, ax = plt.subplots(figsize=(10, 9));

#ncounts_families.display_field(band=2,origin='Lower',title='family richness')
plt.imshow(qh_presences_of_something.rasterdata.bands[1].data(),origin='Lower',cmap=plt.cm.Greens,clim=(0,1))
plt.colorbar(orientation='horizontal')
plt.title("Presences quantile 0.975")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 9));

#ncounts_families.display_field(band=2,origin='Lower',title='family richness')
plt.imshow(prob5.rasterdata.bands[1].data(),origin='Lower',cmap=plt.cm.RdBu,clim=(0,1))
plt.colorbar(orientation='horizontal')
plt.title("Probability of probability more than 0.5 of presence of Burseracea family")

## Export results to GEotif!

In [ ]:
name = "sample_root"
ql_presences_of_something.exportToGeoTiff("ql_"+name)


In [ ]:
mean_presences_of_something.exportToGeoTiff("mean_"+name)

In [ ]:
qh_presences_of_something.exportToGeoTiff("qh_"+name)

In [ ]:
prob5.exportToGeoTiff("prob05"+name)